In [3]:
import json
import numpy as np
import pyvista as pv
from vtk import VTK_TETRA, VTK_TRIANGLE  # Import the correct VTK constant for tetrahedra
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

In [4]:
import os
log_dir = 'operation_log_2025-03-27_16-39-31'
missing_f_bd = []

for filename in os.listdir(log_dir):
    if filename.startswith('operation_log_'):
        file_path = os.path.join(log_dir, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)
            if 'F_bd_before' not in data:
                missing_f_bd.append(filename)

print("The following files are missing F_bd_before:")
for f in missing_f_bd:
    print(f)

The following files are missing F_bd_before:


In [5]:
# Load the data from the JSON file
# file_path = 'operation_log_sec/operation_log_10.json'
file_path = 'operation_log_sec/operation_log_10.json'
# file_path = 'mesh_data_in.json'
with open(file_path, 'r') as f:
    data = json.load(f)
# Check if all operation_log files contain F_bd_before
import os


T_before = np.array(data['T_before']['values'], dtype=np.int32)
V_before = np.array(data['V_before']['values'], dtype=np.float64)
T_after = np.array(data['T_after']['values'], dtype=np.int32)
V_after = np.array(data['V_after']['values'], dtype=np.float64)
F_bd_before = np.array(data['F_bd_before']['values'], dtype=np.int32)
F_bd_after = np.array(data['F_bd_after']['values'], dtype=np.int32)

In [6]:
# Extract all triangles from the tetrahedral mesh excluding vertex 0
def extract_surface_without_vertex(T, V, vertex_to_remove=0):
    """
    Extract surface triangles from a tetrahedral mesh that don't contain the specified vertex
    
    Parameters:
    T - Tetrahedral mesh connectivity
    V - Vertex coordinates
    vertex_to_remove - Index of the vertex to remove
    
    Returns:
    F0 - Surface triangles without the specified vertex
    """
    # Store surface triangles that don't contain the specified vertex
    surface_triangles = []
    
    # Iterate through all tetrahedra
    for i in range(len(T)):
        tet = T[i]
        
        # If tetrahedron contains the vertex to remove, extract faces without that vertex
        if vertex_to_remove in tet:
            # Find the face that doesn't include the vertex to remove
            face_indices = []
            for j in range(4):
                if tet[j] != vertex_to_remove:
                    face_indices.append(tet[j])
            
            # If we found 3 vertices (a complete face), add to results
            if len(face_indices) == 3:
                surface_triangles.append(face_indices)
        else:
            # If tetrahedron doesn't contain the vertex to remove, extract all faces
            faces = [
                [tet[0], tet[1], tet[2]],
                [tet[0], tet[1], tet[3]],
                [tet[0], tet[2], tet[3]],
                [tet[1], tet[2], tet[3]]
            ]
            surface_triangles.extend(faces)
    
    # Convert to NumPy array
    F0 = np.array(surface_triangles)
    
    return F0

In [7]:
# Extract surface triangles without vertex 0
F0 = extract_surface_without_vertex(T_after, V_after)

# Visualize the result
import pyvista as pv
from pyvista import themes

# Set theme
my_theme = themes.DocumentTheme()
pv.set_plot_theme(my_theme)

# Create mesh
mesh = pv.PolyData(V_after, np.hstack([np.full((len(F0), 1), 3), F0]))

# Create plotter
plotter = pv.Plotter()
plotter.add_mesh(mesh, show_edges=True, color='lightblue', opacity=0.7)
plotter.add_points(V_after, color='red', point_size=10)

# Highlight vertex 0
plotter.add_points(V_after[0].reshape(1, -1), color='green', point_size=15)

# Show result
plotter.show()


Widget(value='<iframe src="http://localhost:50365/index.html?ui=P_0x32ed4f890_0&reconnect=auto" class="pyvista…

In [8]:
def harmonic_parameterization(V, F):
    # Use igl.remove_unreferenced to remove unreferenced vertices
    NV, NF, IM, J = igl.remove_unreferenced(V, F)

    # Update mesh data
    V_clean = NV
    F_clean = NF

    # Find boundary loop
    bnd = igl.boundary_loop(F_clean)

    # Map boundary to circle while preserving edge proportions
    bnd_uv = igl.map_vertices_to_circle(V_clean, bnd)

    # Create a dummy vertex array
    V_dummy = np.zeros_like(V_clean)
    
    # Harmonic parameterization for interior vertices
    uv = igl.harmonic(V_clean, F_clean, bnd, bnd_uv, 1)

    return uv, IM, V_clean, F_clean

In [9]:
uv, IM, V_clean, F_clean = harmonic_parameterization(V_after, F0)
print(F_clean)
F = np.copy(F_clean)
# print(F)
# print(igl.boundary_loop(F))
F_list = F.tolist()
F_list.append(igl.boundary_loop(F).tolist())
f0 = F_list.pop(0)

print(F_list)
print(f0)

[[ 0  1  2]
 [ 3  4  5]
 [ 3  6  7]
 [ 8  6  3]
 [ 3  7  4]
 [ 2  9 10]
 [10  3  5]
 [ 9  3 10]
 [ 1  8  9]
 [ 0  8  1]
 [ 9  8  3]
 [ 2  1  9]
 [ 8  0  6]]
[[3, 4, 5], [3, 6, 7], [8, 6, 3], [3, 7, 4], [2, 9, 10], [10, 3, 5], [9, 3, 10], [1, 8, 9], [0, 8, 1], [9, 8, 3], [2, 1, 9], [8, 0, 6], [0, 6, 7, 4, 5, 10, 2]]
[0, 1, 2]


In [18]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

def tutte_embedding_case3(F, f0):
    """
    Compute the Tutte embedding and boundary edge stresses for a planar graph
    with a triangular outer face f0.
    
    Parameters:
      F: list of faces (each a list of vertex IDs), excluding the outer face.
      f0: list of 3 vertex IDs defining the outer triangle.
    
    Returns:
      uv: dict mapping vertex ID -> (x, y) coordinate.
      stress: dict mapping edge (i, j) -> ω_{ij}, symmetric.
    """
    # 1. Collect all vertices and index them 0..n-1
    verts = sorted(set(v for face in F for v in face) | set(f0))
    vid2idx = {v:i for i,v in enumerate(verts)}
    idx2vid = {i:v for v,i in vid2idx.items()}
    n = len(verts)

    # 2. Boundary indices
    B = [vid2idx[v] for v in f0]
    Bset = set(B)

    # 3. Count interior edges from F
    edge_count = {}
    for face in F:
        m = len(face)
        for k in range(m):
            a = vid2idx[face[k]]
            b = vid2idx[face[(k+1)%m]]
            e = (min(a,b), max(a,b))
            edge_count[e] = edge_count.get(e, 0) + 1

    # 4. Build Laplacian L with ω=1 on interior edges
    L = sp.lil_matrix((n, n))
    for (i, j), c in edge_count.items():
        w = 1.0
        L[i, i] += w
        L[j, j] += w
        L[i, j] -= w
        L[j, i] -= w
    L = L.tocsc()

    # 5. Partition into boundary (B) and interior (I)
    I = [i for i in range(n) if i not in Bset]
    LIB = L[I, :][:, B]   # shape (|I|, 3)
    LII = L[I, :][:, I]   # shape (|I|, |I|)

    # 6. Fix outer triangle positions
    pB = np.array([[0.0, 0.0],
                   [1.0, 0.0],
                   [0.0, 1.0]], dtype=float)
    xB = pB[:, 0]
    yB = pB[:, 1]

    # 7. Solve for interior coordinates
    xI = spsolve(LII, -LIB.dot(xB))
    yI = spsolve(LII, -LIB.dot(yB))

    # 8. Assemble uv mapping
    uv = {}
    # boundary
    for idx, v in enumerate(f0):
        uv[v] = (pB[idx, 0], pB[idx, 1])
    # interior
    for idx, i in enumerate(I):
        uv[idx2vid[i]] = (xI[idx], yI[idx])

    # 9. Compute substitution matrix S = LIB^T * (LII^{-1} * LIB)
    X = spsolve(LII, LIB.toarray())
    S = LIB.T.dot(X)  # shape 3×3

    # 10. Assemble stress dict
    stress = {}
    # interior edges
    for (a, b), c in edge_count.items():
        va, vb = idx2vid[a], idx2vid[b]
        stress[(va, vb)] = stress[(vb, va)] = 1.0
    # boundary edges (loop of f0)
    for p in range(3):
        q = (p + 1) % 3
        i, j = B[p], B[q]
        va, vb = idx2vid[i], idx2vid[j]
        wb = -S[p, q]
        stress[(va, vb)] = stress[(vb, va)] = wb

    return uv, stress



In [19]:
uv, stress = tutte_embedding_case3(F_list, f0)
print("Tutte embedding coordinates:")
for v, (x,y) in uv.items():
    print(f"  vertex {v}: ({x:.4f}, {y:.4f})")
uv_array = np.array([uv[v] for v in sorted(uv.keys())])


Tutte embedding coordinates:
  vertex 0: (0.0000, 0.0000)
  vertex 1: (1.0000, 0.0000)
  vertex 2: (0.0000, 1.0000)
  vertex 3: (0.2927, 0.3537)
  vertex 4: (0.2764, 0.3618)
  vertex 5: (0.2683, 0.4214)
  vertex 6: (0.2358, 0.2154)
  vertex 7: (0.2683, 0.3103)
  vertex 8: (0.3821, 0.1978)
  vertex 9: (0.3821, 0.4201)
  vertex 10: (0.2358, 0.5488)


In [20]:
import pyvista as pv
import numpy as np
plotter = pv.Plotter()
uv_3d = np.hstack((uv_array, np.zeros((uv_array.shape[0], 1))))
max_vertices = max(len(face) for face in F_list)
faces = []
for face in F_list: 
    faces.append(len(face))
    faces.extend(face)
mesh = pv.PolyData(uv_3d, faces)
plotter.add_mesh(mesh, color='white', line_width=1, show_edges=True)

# Visualize each edge's stress
for (va, vb), stress_value in stress.items():
    point_a = np.array([uv[va][0], uv[va][1], 0])
    point_b = np.array([uv[vb][0], uv[vb][1], 0])
    midpoint = (point_a + point_b) / 2
    plotter.add_point_labels(midpoint.reshape(1, 3), [f"{stress_value:.2f}"], point_size=8, font_size=10, text_color='blue')

for vid, (x, y) in enumerate(uv_array): 
    plotter.add_point_labels(np.array([[x, y, 0]]), [str(vid)], point_size=10, font_size=12, text_color='red')

plotter.show()



Widget(value='<iframe src="http://localhost:50365/index.html?ui=P_0x3391dde10_3&reconnect=auto" class="pyvista…

In [32]:
import numpy as np
import scipy.sparse as sp
from collections import defaultdict, deque

def compute_lifting(F, f0, uv, stresses):
    """
    Perform Maxwell-Cremona lifting given:
      F: list of faces (each a list of vertex IDs), excluding the outer face f0
      f0: list of 3 vertex IDs defining the outer triangle (boundary face)
      uv: dict mapping vertex ID -> (x, y) from Tutte embedding
      stresses: dict mapping edge (i, j) -> stress value

    Returns:
      xyz: dict mapping vertex ID -> (x, y, z)
    """
    # 1. Build combined face list: pick first F[0] as interior base f1
    F_all = [F[0]] + F[1:] + [f0]
    num_faces = len(F_all)
    
    # 2. Build edge->faces adjacency
    edge2faces = defaultdict(list)
    for fid, face in enumerate(F_all):
        m = len(face)
        for i in range(m):
            a, b = face[i], face[(i+1)%m]
            edge = tuple(sorted((a,b)))
            edge2faces[edge].append(fid)
    
    # 3. Build face adjacency list
    adj_faces = [[] for _ in range(num_faces)]
    for edge, fids in edge2faces.items():
        if len(fids) == 2:
            f1, f2 = fids
            adj_faces[f1].append(f2)
            adj_faces[f2].append(f1)
    
    # 4. Initialize plane params a (2-vector) and d (scalar)
    a = [None] * num_faces
    d = [None] * num_faces
    f_base = num_faces - 2  # last of F_all
    a[f_base] = np.array([0.0, 0.0])
    d[f_base] = 0.0
    
    # 5. BFS over faces starting from base
    queue = deque([f_base])
    visited = {f_base}
    while queue:
        fr = queue.popleft()
        ar, dr = a[fr], d[fr]
        # for each neighbor face
        for fl in adj_faces[fr]:
            if fl in visited:
                continue
            # find common edge
            shared = set(F_all[fr]) & set(F_all[fl])
            # find edge vertices i,j in common
            for edge in [(u,v) for u in shared for v in shared if u< v]:
                if edge in edge2faces:
                    fids = edge2faces[edge]
                    if fr in fids and fl in fids:
                        i, j = edge
                        break
            # determine orientation: want fl left of directed (i->j)
            pi, pj = np.array(uv[i]), np.array(uv[j])
            # find third vertex k of fl
            k = next(v for v in F_all[fl] if v not in edge)
            pk = np.array(uv[k])
            # cross of (pj-pi) x (pk-pi)
            cross = np.cross(pj-pi, pk-pi)
            if cross < 0:
                # swap to make fl on left
                i, j = j, i
                pi, pj = pj, pi
            # stress on this edge
            wij = stresses[(i,j)]
            # compute (pi - pj)_perp
            v = pi - pj
            v_perp = np.array([-v[1], v[0]])
            # compute a[fl] and d[fl]
            a[fl] = wij * v_perp + ar
            # perp of pj
            pj_perp = np.array([-pj[1], pj[0]])
            d[fl] = wij * np.dot(pi, pj_perp) + dr
            visited.add(fl)
            queue.append(fl)
    
    # 6. Compute z for each vertex
    # build vertex->incident-face map
    vert2faces = defaultdict(list)
    for fid, face in enumerate(F_all):
        for v in face:
            vert2faces[v].append(fid)
    xyz = {}
    for v, (x,y) in uv.items():
        # pick first incident face
        fid = vert2faces[v][0]
        z = np.dot(a[fid], [x,y]) + d[fid]
        xyz[v] = (x, y, z)
    
    return xyz




In [33]:
xyz = compute_lifting(F_list, f0, uv, stress)
print(xyz)

# Visualize F_list and xyz
plotter = pv.Plotter()
# Convert xyz to a format suitable for pyvista
points = np.array([xyz[v] for v in sorted(xyz.keys())])
faces = []
for face in F_list:
    faces.append(len(face))
    faces.extend(face)
# Create polygonal data
mesh = pv.PolyData(points, faces)
# Add mesh to the plotter
plotter.add_mesh(mesh, color='white', line_width=1, show_edges=True)
# Add labels for each vertex
for vid, (x, y, z) in xyz.items():
    plotter.add_point_labels(np.array([[x, y, z]]), [str(vid)], point_size=10, font_size=12, text_color='red')
# Show the plot
plotter.show()

{0: (np.float64(0.0), np.float64(0.0), np.float64(0.0)), 1: (np.float64(1.0), np.float64(0.0), np.float64(-0.41327913279132816)), 2: (np.float64(0.0), np.float64(1.0), np.float64(0.0)), 3: (np.float64(0.2926829268292682), np.float64(0.3536585365853657), np.float64(-0.0009033423667570047)), 4: (np.float64(0.2764227642276422), np.float64(0.36178861788617867), np.float64(-6.938893903907228e-18)), 5: (np.float64(0.2682926829268292), np.float64(0.4214092140921407), np.float64(-6.938893903907228e-18)), 6: (np.float64(0.23577235772357721), np.float64(0.21544715447154464), np.float64(1.734723475976807e-18)), 7: (np.float64(0.26829268292682923), np.float64(0.31029810298102967), np.float64(3.469446951953614e-18)), 8: (np.float64(0.38211382113821135), np.float64(0.19783197831978314), np.float64(-0.03568202348690151)), 9: (np.float64(0.38211382113821135), np.float64(0.4200542005420053), np.float64(-0.03568202348690158)), 10: (np.float64(0.2357723577235772), np.float64(0.5487804878048779), np.float

/var/folders/3r/4zk6q98x5zv62659rgk70q240000gn/T/ipykernel_10903/274959743.py:69: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  cross = np.cross(pj-pi, pk-pi)


Widget(value='<iframe src="http://localhost:50365/index.html?ui=P_0x35a8e75d0_10&reconnect=auto" class="pyvist…

In [23]:
# Calculate the normal vector and center of the last face
last_face = F_list[-1]
print("Last face:", last_face)

# Calculate the center of the polygon
center = np.mean([np.array(xyz[v]) for v in last_face], axis=0)
print("Polygon center:", center)

# Calculate the normal vector of the polygon
p1 = np.array(xyz[last_face[0]])
p2 = np.array(xyz[last_face[1]])
p3 = np.array(xyz[last_face[2]])
normal = np.cross(p2 - p1, p3 - p1)
if np.linalg.norm(normal) != 0:
    normal = normal / np.linalg.norm(normal)  # Normalize the normal vector
else:
    print("Warning: The magnitude of the normal vector is zero, cannot normalize")
    normal = np.array([0, 0, 1])  # Default normal vector is the z-axis

print(normal)

# Calculate the rotation matrix to align the normal vector with the z-axis
z_axis = np.array([0, 0, 1])
v = np.cross(normal, z_axis)
c = np.dot(normal, z_axis)
if 1 - c**2 != 0:
    k = np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])
    rotation_matrix = np.eye(3) + k + k @ k * ((1 - c) / (1 - c**2))
else:
    print("Warning: Denominator is zero in rotation matrix calculation, using identity matrix")
    rotation_matrix = np.eye(3)
print("Rotation matrix:", rotation_matrix)

# Rotate and translate xyz
xyz_rotated_translated = {}
for v, (x, y, z) in xyz.items():
    point = np.array([x, y, z]) - center  # Translate to origin
    rotated_point = rotation_matrix @ point  # Rotate
    xyz_rotated_translated[v] = rotated_point

# Update xyz with the rotated and translated coordinates
xyz = xyz_rotated_translated
print("Updated xyz coordinates:", xyz)

Last face: [0, 6, 7, 4, 5, 10, 2]
Polygon center: [ 0.18350755  0.40824623 -0.00516196]
[-0.05951295 -0.0081154   0.99819454]
Rotation matrix: [[ 9.98227504e-01 -2.41703989e-04  5.95129539e-02]
 [-2.41703989e-04  9.99967040e-01  8.11540280e-03]
 [-5.95129539e-02 -8.11540280e-03  9.98194544e-01]]
Updated xyz coordinates: {0: array([-1.83932257e-01, -4.08304140e-01, -5.59204493e-17]), 1: array([ 0.79324798, -0.41141593, -0.41253298]), 2: array([-1.83690116e-01,  5.91728879e-01,  1.51743403e-16]), 3: array([ 0.10930226, -0.05457039, -0.00090171]), 4: array([ 9.30689553e-02, -4.64366428e-02, -2.89298179e-17]), 5: array([ 8.49388740e-02,  1.31839534e-02, -1.50520301e-17]), 6: array([ 5.23109305e-02, -1.92792781e-01, -4.10390549e-17]), 7: array([ 8.49119694e-02, -9.79308265e-02, -3.89983731e-17]), 8: array([ 0.19678444, -0.21066268, -0.0356176 ]), 9: array([ 0.19683825,  0.01156688, -0.0356176 ]), 10: array([5.23916442e-02, 1.40551558e-01, 2.34250503e-17])}


In [27]:
# 将所有xyz的z坐标乘以3
for v in xyz:
    xyz[v][2] *= 10


In [28]:
# Visualize F_list and xyz
plotter = pv.Plotter()
# Convert xyz to a format suitable for pyvista
points = np.array([xyz[v] for v in sorted(xyz.keys())])
faces = []
for face in F_list:
    faces.append(len(face))
    faces.extend(face)
# Create polygonal data
mesh = pv.PolyData(points, faces)
# Add mesh to the plotter
plotter.add_mesh(mesh, color='white', line_width=1, show_edges=True)
# Add labels for each vertex
for vid, (x, y, z) in xyz.items():
    plotter.add_point_labels(np.array([[x, y, z]]), [str(vid)], point_size=10, font_size=12, text_color='red')
# Show the plot
plotter.show()


Widget(value='<iframe src="http://localhost:50365/index.html?ui=P_0x349539910_7&reconnect=auto" class="pyvista…

In [29]:
V_param = np.zeros_like(V_after)
for i in range(len(IM)):
    if IM[i] != -1:
        V_param[i] = xyz[IM[i]]

V_param[0] = np.zeros(3)

V_param0 = np.copy(V_param)
V_param0[0] = V_param[5]

my_theme = themes.DocumentTheme()
pv.set_plot_theme(my_theme)

plotter = pv.Plotter()

cells = np.hstack([4*np.ones((len(T_after), 1), dtype=np.int32), T_after])
cell_types = np.full(len(T_after), 10, dtype=np.int32)
tet_mesh = pv.UnstructuredGrid(cells, cell_types, V_param)

all_faces = []
tet_face_map = {}
for i, tet in enumerate(T_after):
    faces = [
        tuple(sorted([tet[0], tet[1], tet[2]])),
        tuple(sorted([tet[0], tet[1], tet[3]])),
        tuple(sorted([tet[0], tet[2], tet[3]])),
        tuple(sorted([tet[1], tet[2], tet[3]]))
    ]
    for face in faces:
        if face not in tet_face_map:
            tet_face_map[face] = []
        tet_face_map[face].append(i)
        all_faces.append(face)

internal_face = None
tet_pair = None
internal_faces = []
for face, tets in tet_face_map.items():
    if len(tets) == 2:
        internal_faces.append((list(face), tets))

if len(internal_faces) > 0:
    idx = np.random.randint(0, len(internal_faces))
    # idx = 11
    internal_face, tet_pair = internal_faces[idx]

if internal_face and tet_pair:
    plotter.add_mesh(tet_mesh, style='wireframe', opacity=0.3, color='gray', line_width=2.5)
    
    for tet_idx in tet_pair:
        tet_vertices = V_param[T_after[tet_idx]]
        tet_cells = np.array([[4, 0, 1, 2, 3]])
        single_tet = pv.UnstructuredGrid(tet_cells, [10], tet_vertices)
        # plotter.add_mesh(single_tet, opacity=0.3, color='grey')
    for tet_idx in tet_pair:
        tet = T_after[tet_idx]
        if 0 not in tet[1:]:
            tri_vertices = V_param[tet[1:]]
            tri_cells = np.array([[3, 0, 1, 2]])
            tri_mesh = pv.PolyData(tri_vertices, tri_cells)
            # 使用不同的颜色
            color = 'blue' if tet_idx % 2 == 0 else 'green'
            plotter.add_mesh(tri_mesh, color=color, opacity=0.8)
    internal_tri = np.array([3] + internal_face)
    tri_mesh = pv.PolyData(V_param, internal_tri.reshape(1,-1))

    # tri_mesh0 = pv.PolyData(V_param0, internal_tri.reshape(1,-1))
    plotter.add_mesh(tri_mesh, color='red', opacity=0.8)
    # plotter.add_mesh(tri_mesh0, color='red', opacity=0.8)

    plotter.add_points(V_param, color='black', point_size=8)
    
    for i, point in enumerate(V_param):
        plotter.add_point_labels(
            point.reshape(1,-1), 
            [str(i)], 
            point_size=20,
            font_size=14,
            always_visible=True)
plotter.show()


Widget(value='<iframe src="http://localhost:50365/index.html?ui=P_0x34870f0d0_8&reconnect=auto" class="pyvista…

In [224]:
V_param

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.83932257e-01, -4.08304140e-01, -5.59204493e-17],
       [ 7.93247983e-01, -4.11415926e-01, -4.12532976e-01],
       [-1.83690116e-01,  5.91728879e-01,  1.51743403e-16],
       [ 1.09302262e-01, -5.45703863e-02, -9.01711422e-04],
       [ 9.30689553e-02, -4.64366428e-02, -2.89298179e-17],
       [ 8.49388740e-02,  1.31839534e-02, -1.50520301e-17],
       [ 5.23109305e-02, -1.92792781e-01, -4.10390549e-17],
       [ 8.49119694e-02, -9.79308265e-02, -3.89983731e-17],
       [ 1.96784445e-01, -2.10662678e-01, -3.56176012e-02],
       [ 1.96838254e-01,  1.15668819e-02, -3.56176012e-02],
       [ 5.23916442e-02,  1.40551558e-01,  2.34250503e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [203]:
T_after

array([[ 0,  1,  2,  3],
       [ 0,  4,  5,  6],
       [ 0,  4,  7,  8],
       [ 0,  9,  7,  4],
       [ 0,  4,  8,  5],
       [ 0,  3, 10, 11],
       [ 0, 11,  4,  6],
       [ 0, 10,  4, 11],
       [ 0,  2,  9, 10],
       [ 0,  1,  9,  2],
       [ 0, 10,  9,  4],
       [ 0,  3,  2, 10],
       [ 0,  9,  1,  7]], dtype=int32)